In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import panel as pn

pn.extension('tabulator')

#import cartopy.crs as ccrs
#import geoviews as gv 
#import pyproj

#pn.extension(sizing_mode="stretch_width")

import hvplot.pandas

# Generation of an interactive table

```
pipenv run panel convert notebooks/93_Provide_interactive_table.ipynb --to pyodide-worker --title interactive_table  --out ./html
```

Umbenennen der Dateien in der Ordnerstruktur nicht vergessen! Anpassen des js Pfades in der html Datei analog.

In [2]:
input_data_folder_base = Path("../results/")
input_data_folder = Path.cwd() / input_data_folder_base
overall_collection_file_name = "overall_collection_2023.p"
key_parameter_file_name = "key_parameter_2023.p"
cluster_positions_file_name = "cluster_positions_2023.p"
print(f"Reading raw pandas files from \n    {input_data_folder}")

output_folder_base = Path("../html/")
output_image_folder = output_folder_base / Path("./images/")
print(f"Writing website information to \n    {output_folder_base}")

Reading raw pandas files from 
    /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/RoBiMo_Trop_DataSet/notebooks/../results
Writing website information to 
    ../html


In [3]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():
    #df_key_param = pd.read_pickle(input_data_folder / key_parameter_file_name)
    #df_key_param = pd.read_pickle("https://api.allorigins.win/raw?url=https://github.com/SebastianZug/RoBiMo_Trop_DataSet/blob/main/results/key_parameter_2023.p?raw=true")
    df_key_param = pd.read_pickle("https://sebastianzug.github.io/RoBiMo_Trop_DataSet/results/key_parameter_2023.p?raw=true")
    pn.state.cache['data'] = df_key_param.copy()
else: 
    df_key_param = pn.state.cache['data']

In [4]:
df_key_param

,start,end,experiment_location,position,lat_est_mean,lon_est_mean,hdop_max,nsats_min,CO2_min,CO2_max,humidity_in_mean,humidity_out_mean,airtemp_in_mean,PAR_min,PAR_max,airtemp_out_mean
corresponding_meas_file,,,,,,,,,,,,,,,,
010323-140717-ADC.p,2023-03-01 14:07:17,2023-03-01 14:12:17,Balbina,2.0,-1.903664,-59.460595,0.74,21.0,203.432988,378.423200,94.543262,82.949425,30.826920,754.527032,1311.968820,29.837064
010323-141917-ADC.p,2023-03-01 14:19:17,2023-03-01 14:24:18,Balbina,2.0,-1.903664,-59.460595,0.64,22.0,232.852340,379.521848,94.579406,83.347764,30.890230,806.163488,1068.801396,29.866438
010323-143117-ADC.p,2023-03-01 14:31:17,2023-03-01 14:36:18,Balbina,2.0,-1.903665,-59.460596,0.58,21.0,194.735358,378.453718,94.464311,82.534030,30.967107,742.686048,978.590188,30.071834
010323-144317-ADC.p,2023-03-01 14:43:17,2023-03-01 15:13:49,Balbina,2.0,-1.903664,-59.460595,0.66,18.0,193.636710,374.455860,93.878567,81.218147,31.770508,601.875996,1447.163560,30.523049
010323-213810-ADC.p,2023-03-01 21:38:10,2023-03-01 21:43:10,Balbina,0.0,-1.909374,-59.461053,0.69,18.0,161.318148,367.192576,98.121686,95.432630,28.193526,0.000000,0.000000,26.250800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
070323-135435-ADC.p,2023-03-07 13:54:35,2023-03-07 13:59:35,Caldeirao,3.0,-3.256652,-60.218548,0.65,20.0,343.663198,383.519706,99.798149,98.530668,26.495527,285.953660,316.471660,24.664316
070323-140635-ADC.p,2023-03-07 14:06:35,2023-03-07 14:11:35,Caldeirao,3.0,-3.256654,-60.218538,0.63,22.0,337.590116,376.500566,99.664161,98.496821,26.707622,0.244144,249.393096,24.659334
070323-141835-ADC.p,2023-03-07 14:18:35,2023-03-07 14:23:35,Caldeirao,3.0,-3.256649,-60.218538,0.59,21.0,365.910820,387.548082,99.592889,98.551707,26.676435,0.183108,212.588388,24.660895


In [5]:
# Make DataFrame Pipeline Interactive
idf = df_key_param.interactive()

In [6]:
select_lake = pn.widgets.Select(
    name='Location',
    value="Balbina",
    options=list(df_key_param.experiment_location.unique())
)

select_position = pn.widgets.Select(
    name="Measurement",
    value=df_key_param.position.unique()[0],
    options=df_key_param[df_key_param.experiment_location==select_lake.value].position.unique().tolist()
)

@pn.depends(select_lake.param.value, watch=True)
def _update_lake(select_lake):
    positions = list(df_key_param[df_key_param.experiment_location==select_lake]\
                            .position.unique())
    select_position.options = positions
    select_position.value = positions[0]

In [7]:
relevant_parameter = ['start', 'end', 'hdop_max', 'nsats_min', 'CO2_min', 'CO2_max', 'PAR_min',  'PAR_max']

In [8]:
data_pipeline_overview = (
    idf[
        (idf.experiment_location == select_lake)
    ]\
           [relevant_parameter]\
           .describe(datetime_is_numeric=True)\
           .loc[['mean', 'std', 'min', 'max'],:]
)

# data_pipeline_overview

In [9]:
data_pipeline_all = (
    idf[
        (idf.experiment_location == select_lake ) &
        (idf.position == select_position)
    ]\
        [relevant_parameter]
)

#data_pipeline_all

In [10]:
#co_plot=selected_data_pipeline.hvplot(use_index=True, y='CO2(ppm)', title="CO_2 concentration")
#pressure_plot=selected_data_pipeline.hvplot(use_index=True, y='pressure in(mbar)', title="Pressure in")

In [11]:
table_overview = data_pipeline_overview.pipe(pn.widgets.Tabulator, pagination="remote", title="Aggregated key parameters of the position")
table_all = data_pipeline_all.pipe(pn.widgets.Tabulator, pagination="remote", title="Parameter of all measurements at this position")

In [16]:
#template = pn.template.FastListTemplate(
#    title = "RoBiMo 2023 - Data exploration",
#    sidebar =[pn.pane.Markdown("# Select the relevant lake and measurement"),
#              select_lake,
#              select_file],
#    main=[pn.Row(
#            pn.Column(
#               position_plot.panel(width=500, margin=(0,25))
#            ),
#            pn.Column(
#               table.panel(width=1000, margin=(0,25))
#            )
#         ),
#         pn.Row(
#            pn.Column(
#               pressure_plot.panel(width=600, margin=(0,25))
#            ),
#            pn.Column(
#               co_plot.panel(width=600, margin=(0,25))
#            )
#         )     
#    ]
#)

template = pn.template.FastListTemplate(
    title = "RoBiMo 2023 - Data overview",
    sidebar =[pn.pane.Markdown("# Selection"),
              select_lake,
              select_position],
    main=[pn.Row(
               pn.pane.Markdown("__Aggregated key parameters of the lake__"),
               table_overview.panel(width=1000)
         ),
         pn.Row(
               pn.pane.Markdown("__Parameter of all measurements at this position__"),
               table_all.panel(width=1000)
         )     
    ]
)

#template.show()
template.servable();

Launching server at http://localhost:39841
